<a href="https://colab.research.google.com/github/gondore/nsdc-crafter/blob/main/Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Replay Memory**

Replay memory typically stores tuples (state, action, reward, next_state) to leverage past training experience

__init__ initializes the memory with max length, which is necessary for memory management

__store_transition__ 'pushes' a transition in the memory.

In [1]:
# INPUT
import random
from collections import deque

class ReplayMemory:
    def __init__(self, capacity):
        # initialize a new deque with a fixed size as the memory.
        # this holds the transitions/tuples.
        # HINT: Use deque(maxlen=capacity) to ensure the memory does not grow beyond the set capacity.


    def store_transition(self, state, action, reward, next_state, done):
        # this 'pushes' tuples into memory.


    def sample(self, batch_size):
        # randomly sample a batch of transitions from the memory.
        # ensure the batch size does not exceed the length of the memory.
        # hint: Use random.sample(self.memory, batch_size) to get a random subset.


    def __len__(self):
        # Return the current size of the memory.
        # this is used to determine when there's enough samples to begin training


# **Action Selection**
this is the class to describe how the agent decides to exploit or explore.
__select_action__ updates epsilon using a decay formulam and selects an action.
exloit:
- converting current state to a tensor formatted for the policy network
- passing the state tensor through the policy network to get q-values for all possible actions
- finally selecting the action with the max q-value
input: state and step
explore:
- randomly select an action
output:

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

class ActionSelector:
    def __init__(self, q_network, epsilon_start, epsilon_end, epsilon_decay):

        #tracks number of actions taken
        self.current_step = 0
        # TODO: Initialize the parameters for epsilon greedy strategy
        # this will be epsilon_start, epsilon_end, epsilon_decay, num_actions, policy_net, and epsilon
        # examples below
        self.epsilon = epsilon_start  # Starting value of epsilon
        self.epsilon = epsilon_start        # Initialize epsilon with epsilon_start
        self.epsilon_end = epsilon_end  # Minimum value of epsilon

    # TODO: implement the epsilon-greedy action selection strategy.
    def select_action(self, state, step):
        self.current_step += 1

        # TODO: Update the value of epsilon using the decay rate formula.
        # new epsilon = end + (epsilon start - epsilon end) * (current step / epsilon decay)**(-1)
        self.epsilon =

        # If the sample is greater than epsilon, choose the best action (exploit).
        if np.random.rand() > self.epsilon:
          state = #convert state to tensor and ensure data type float 32
          with torch.no_grad():
              state_tensor = torch.FloatTensor(state).unsqueeze(0)  # Convert state to tensor and add batch dimension
              action_values = self.q_network(state_tensor)  # Get action values from the Q-network
              action = torch.argmax(action_values).item()  # Choose the action with the highest value
              q_values = self.policy_net(state) # this updates the qvalues
              return q_values.max(1)[1].view(1, 1).item()
        else:
            # TODO: else select a random action (explore) by returning a number to represent the action
            action =
        return action


# **CNN**

- takes preprocessed image sequneces as input and outputs the q values for each possible action.

In [ ]:
import torch
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, input_shape, num_actions):
        super(CNN, self).__init__()
        self.input_shape = input_shape
        self.num_actions = num_actions
        # Define your convolutional layers here

    def forward(self, x):
        # Implement the forward pass
        return x

    def preprocess(self, raw_state):
        # Preprocess the raw state (image) received from the environment
        return raw_state


# **main**
- this will import everything and start training

In [2]:
import torch
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
import numpy as np
import gym
import crafter
from ReplayMemory import ReplayMemory
from ActionSelector import ActionSelector
from CNN import CNN

# Hyperparameters
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10
NUM_EPISODES = 50
LEARNING_RATE = 1e-3
MEMORY_SIZE = 10000

# Environment setup
env = crafter.env
env.seed(42)

# Get number of actions from gym action space
n_actions = env.action_space.n

# Get screen size from gym observation space
_, _, screen_height, screen_width = env.observation_space.shape

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Neural network and optimizer setup
policy_net = CNN(screen_height, screen_width, n_actions).to(device)
target_net = CNN(screen_height, screen_width, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()  # Set the target net to evaluation mode

optimizer = optim.RMSprop(policy_net.parameters(), lr=LEARNING_RATE)
memory = ReplayMemory(MEMORY_SIZE)
action_selector = ActionSelector(policy_net, n_actions, EPS_START, EPS_END, EPS_DECAY)

# Transformation pipeline
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((screen_height, screen_width)),
    transforms.ToTensor()
])

# Training loop
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    batch = tuple(zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                            batch[3])), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch[3] if s is not None]).to(device)

    state_batch = torch.cat(batch[0]).to(device)
    action_batch = torch.cat(batch[1]).to(device)
    reward_batch = torch.cat(batch[2]).to(device)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the columns of actions taken
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()

    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

for episode in range(NUM_EPISODES):
    # Initialize the environment and state
    state = env.reset()
    state = transform(state).unsqueeze(0).to(device)

    for t in range(1000):  # run for a maximum of 1000 steps
        # Select and perform an action
        action = action_selector.select_action(state, device)
        next_state, reward, done, _ = env.step(action)
        reward = torch.tensor([reward], device=device)

        # Observe new state
        if not done:
            next_state = transform(next_state).unsqueeze(0).to(device)
        else:
            next_state = None

        # Store the transition in memory
        memory.push(state, torch.tensor([[action]], device=device), reward, next_state)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()
        if done:
            break

    # Update the target network, copying all weights and biases in DQN
    if episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')
env.close()

# Save the trained model
torch.save(policy_net.state_dict(), 'crafter_dqn.pth')


ModuleNotFoundError: No module named 'crafter'